In [29]:
import torch
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision

In [30]:
class Generator(nn.Module):
    def __init__(self,z_dimension, number_features, number_channels, number_of_classes):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=z_dimension + number_of_classes, out_channels=number_features*4, kernel_size=4, stride=1, padding=0), #100x1x1 to 256x4x4
            nn.BatchNorm2d(number_features*4),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(in_channels=number_features*4, out_channels=number_features*2, kernel_size=4, stride=2, padding=1), # 256x4x4 to 128x8x8
            nn.BatchNorm2d(number_features*2),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(in_channels=number_features*2, out_channels=number_features, kernel_size=4, stride=2, padding=1), # 128x8x8 to 64x16x16
            nn.BatchNorm2d(number_features*2),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(in_channels=number_features, out_channels=number_channels, kernel_size=4, stride=2, padding=1), # 64x16x16 to 1x32x32
            nn.Tanh()
        )

    def forward(self, z, labels):
        z = torch.cat([z, labels], dim=1)
        return z
    

class Discriminator(nn.Module):
    def __init__(self, number_disc_features, number_channels, number_of_classes):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=number_channels + number_of_classes, out_channels=number_disc_features, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=number_disc_features, out_channels=number_disc_features * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(number_disc_features * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=number_disc_features * 2, out_channels=number_disc_features * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(number_disc_features * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels=number_disc_features * 4, out_channels=number_channels, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input, labels):
        x = torch.cat([input, labels], dim=1)
        return self.main(x)
    

In [31]:
# GAN loss function according to the formula
def gan_loss(D, G, real_data, latent_dim, device):
    batch_size = real_data.size(0)
    
    # Labels for real and fake data
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)
    
    # Loss function
    criterion = nn.BCELoss()
    
    # Forward pass real batch through Discriminator
    D_real = D(real_data)
    real_loss = criterion(D_real, real_labels)
    
    # Generate fake data
    z = torch.randn(batch_size, latent_dim).to(device)
    fake_data = G(z)
    
    # Forward pass fake batch through Discriminator
    D_fake = D(fake_data)
    fake_loss = criterion(D_fake, fake_labels)
    
    # Total Discriminator loss
    D_loss = real_loss + fake_loss
    
    # Generator loss
    G_loss = criterion(D_fake, real_labels)
    
    return D_loss, G_loss

In [32]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])
# Download Fashion MNIST dataset
fashion_mnist_dataset = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print(device)

# Load Fashion-MNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

size_of_feature_map_generator = 64
size_of_feature_map_discriminator = 64
image_channels = 1
latent_dimension = 100
lr = 0.0001
# Training Loop
num_epochs = 20
criterion = nn.BCELoss()

generator = Generator(latent_dimension, size_of_feature_map_generator, image_channels, 10).to(device)
discriminator = Discriminator(size_of_feature_map_discriminator, image_channels, 10).to(device)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))


for epoch in range(num_epochs):
    for i, (real_images, labels) in enumerate(train_loader):
        batch_size = real_images.size(0)
        
        ############################
        # Train Discriminator
        ###########################
        discriminator.zero_grad()
        
        # Real data
        real_images = real_images.to(device)
        D_real = discriminator(real_images, labels.to(device))
        real_loss = criterion(D_real, torch.ones_like(D_real))
        
        # Fake data
        z = torch.randn(batch_size, latent_dimension, device=device)
        fake_images = generator(z, labels.to(device))
        D_fake = discriminator(fake_images.detach(), labels.to(device))
        fake_loss = criterion(D_fake, torch.zeros_like(D_fake))
        
        # Total discriminator loss
        D_loss = real_loss + fake_loss
        D_loss.backward()
        optimizer_discriminator.step()
        
        ############################
        # Train Generator
        ###########################
        generator.zero_grad()
        
        # Generate fake images
        z = torch.randn(batch_size, latent_dimension, device=device)
        fake_images = discriminator(z, labels.to(device))
        
        # Pass fake images through discriminator
        D_fake = generator(fake_images, labels.to(device))
        
        # Generator loss
        G_loss = criterion(D_fake, torch.ones_like(D_fake))
        G_loss.backward()
        optimizer_generator.step()
        
        # Output training stats
        if i % 100 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Step [{i}/{len(train_loader)}], '
                  f'D_Loss: {D_loss.item():.4f}, G_Loss: {G_loss.item():.4f}')

# Save models after training
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')


cuda:0


DeferredCudaCallError: CUDA call failed lazily at initialization with error: module 'torch' has no attribute 'version'

CUDA call was originally invoked at:

  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Ilir\anaconda3\envs\gen-ai\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Ilir\anaconda3\envs\gen-ai\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Ilir\anaconda3\envs\gen-ai\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\Ilir\anaconda3\envs\gen-ai\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "c:\Users\Ilir\anaconda3\envs\gen-ai\Lib\asyncio\base_events.py", line 607, in run_forever
    self._run_once()
  File "c:\Users\Ilir\anaconda3\envs\gen-ai\Lib\asyncio\base_